<a href="https://colab.research.google.com/github/T2718/AI_Make_Image/blob/main/CivitAI/output_del-v16-GUI-Mixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Driveのマウント
#@markdown **Driveを使用する場合はチェックを入れて実行**
drive_use = True #@param {type:"boolean"}
mount_point = '/content/drive'

#drive_use = Drive_path.strip() != ""
if drive_use:
  from google.colab import drive
  import os
  if not os.path.exists(mount_point):
    drive.mount(mount_point)
  if not os.path.exists(mount_point):
    drive_use = False
    print("Google Driveに接続できません。")

In [21]:
#@title 変数の設定
#@markdown Google Driveに生成した画像を保存したい場合は<br>
#@markdown そのディレクトリのパスを入力。
#@markdown <br>Driveを使用しない際は空欄のまま<br>
#@markdown (例:/content/drive/MyDrive/Colab Notebooks/Generated_IMG)
Drive_path = "/content/drive/MyDrive/Colab Notebooks/share_dir/AI_IMG/generated" #@param {type:"string"}
#drive_use = Drive_path.strip() != ""
if drive_use:
  drive_save = Drive_path
else:
  drive_save = ""
#@markdown ##種類
#@markdown 1. **Hugging Face**
#@markdown 2. **CivitAI**<br>
#@markdown    [1,2について]Text to ImageまたはImage to Imageを<br>
#@markdown    行うためのモデルの種類
#@markdown 3. **トークン数確認**
#@markdown   文のトークン数を確認できる。
#@markdown 4. **ダウンロード**
#@markdown 5. **削除**<br>
#@markdown   [4,5について]<small>*※Google Driveを使わずに画像生成した場合のみ有効*</small><br>
#@markdown   /content/downloadディレクトリを.zipでダウンロード<br>
#@markdown   または削除
#@markdown 6. **初期値**<br>
#@markdown   フォームの値の初期値を表示<br>
#@markdown   <small>ただし、フォームの値は置き換わりません。</small>
Kind = "CivitAI" #@param ["Hugging Face","CivitAI","Token数確認","ダウンロード","削除","初期値"]
#@markdown ***以下、括弧()内に種類の番号ごとの入力必須の欄を指定***
#@markdown ##Text to Image(t2i)かImage to Image(i2i) (1,2)
SubMode = "t2i" #@param ["t2i","i2i"]
#@markdown ##プロンプト (1,2,3)
Prompt = "anime, 1girl, cute, golden fox ears, golden hair, fluffy fox tail, standing in snowfield, milky way, starry sky, falling snow, glowing particles, fantasy atmosphere, flowing sleeves, hair blowing, dynamic pose, cinematic lighting, looking at viewer, gentle smile, elegant, masterpiece, high quality, modest breasts" #@param {type:"string"}
#@markdown ##ネガティブプロンプト (1,2)
Negative_Prompt = "low quality, worst quality, blurry, bad anatomy, deformed, extra limbs, missing fingers, fused fingers, long neck, bad proportions, ugly face, cropped, watermark, text, flat background, poorly drawn, pixelated, disfigured, low resolution, dark background, unnatural light, boring background" #@param {type:"string"}
#@markdown ##枚数 (1,2)
Num = 20 #@param {type:"number"}
#@markdown ##Step (1,2)
Steps = 200 #@param {type:"number"}
#@markdown ##画像のPath ([1,2]のi2i)
ImagePath = "" #@param {type:"string"}
#@markdown ##画像に従う強度 ([1,2]のi2i)
Strength = 0.7 #@param {type:"number"}
#@markdown ##プロンプトに従う強度 (1,2)
Guidance = 7.5 #@param {type:"number"}
#@markdown ##モデルの状況 (2)
#@markdown - **none** インストールもしていない。
#@markdown - **origin** インストールはしているが<br>
#@markdown   変換はしていない。
#@markdown - **complete** 変換も済んでいる。そのまま使える。
Model_Status = "complete" #@param ["none","origin","complete"]
#@markdown ##Modelの種類<br>
#@markdown *(**1(=Hugging Face)**,2で先程の解答がnoneまたはoriginの時)*
#@markdown - 1(=Hugging Face)
#@markdown   - モデルの種類<br>
#@markdown     <small>以下、例</small>
#@markdown     - stablediffusionapi/eleet-model
#@markdown     - stablediffusionapi/brav6
#@markdown     - Vsukiyaki/ShiratakiMix
#@markdown     - stabilityai/stable-diffusion-xl-base-1.0
#@markdown     - eimiss/EimisAnimeDiffusion_2.0v
#@markdown - 2(=CivitAI)で、先程のモデルの状況で<br>
#@markdown   noneまたはoriginを選択した場合。<br>
#@markdown   <small>以下のどれかに従って入力。</small>
#@markdown   - 特定の数字を入力
#@markdown     1. Anime Pastel Dream<br>
#@markdown     <small>(SDXL:False,NeedYaml:False)</small>
#@markdown     2. WAI-NSFW-illustrious-SDXL v:11.0<br>
#@markdown     <small>(SDXL:True,NeedYaml:True)</small>
#@markdown   - \#◯◯◯◯(数字)<br>
#@markdown     Model ID
#@markdown   - CivitAIモデルのURL(https:~~)
#@markdown   更に、この2つ後の欄(.yaml)も選択必須
Model = "1" #@param {type:"string"}
#@markdown ##SDXLかどうか (2)
Sdxl = False #@param {type:"boolean"}
#@markdown ##.yamlファイルが必要かどうか
#@markdown 三つ前の解答がnoneまたはoriginで、<br>
#@markdown SDXLである場合、選択必須
#@markdown <small>Modelの.safetensorsに.yamlファイルが<br>
#@markdown 使用されていない場合必要。<br>
#@markdown 迷ったら入れてみよう。</small>
NeedYaml = False #@param {type:"boolean"}
#@markdown ##Pipelineをインストールするか<br>
#@markdown ###以下の時はする必要があります。<br>
#@markdown ###基本的に、続けて画像生成を行う際には必要ありません。
#@markdown - モデルの更新を行った時(インストールor変換)
#@markdown - 現セッションで初の生成の時
PipeInstall = False #@param {type:"boolean"}
#@markdown ##ライブラリのインストールをするか<br>
#@markdown 現セッション内で一度でも[1,2,3]のどれかを<br>
#@markdown 実行した場合はライブラリのインストールが必要ありません。
NeedInstall = False #@param {type:"boolean"}
#@markdown ##Loraのパス
#@markdown 使用する際はmodelのURLを入力。
#@markdown 使用しない場合は空欄のまま
LoraPath = "" #@param {type:"string"}



#変数の変換 ------------------------------


# --- Drive設定 ---
#drive_use = Drive_path.strip() != ""

# --- モードを番号に変換 ---
mode = (
  "1" if Kind == "Hugging Face"
  else "2" if Kind == "CivitAI"
  else "3" if Kind == "Token数確認"
  else "4" if Kind == "ダウンロード"
  else "5" if Kind == "削除"
  else "6" if Kind == "初期値"
  else ""
)

# --- パラメータ辞書の初期化 ---
hf_params = {}
civitai_params = {}

# --- Hugging Face ---
if mode == "1":
  hf_params = {
    "need_pipe": PipeInstall,
    "model_id": Model,
    "submode": "2" if SubMode == "i2i" else "1",
    "prompt": Prompt,
    "negative_prompt": Negative_Prompt,
    "num": Num,
    "steps": Steps,
    "guidance": Guidance
  }
  if SubMode == "i2i":
    hf_params["strength"] = Strength
    hf_params["image_path"] = ImagePath

# --- CivitAI ---
elif mode == "2":
  civitai_params = {
    "status": (
      "1" if Model_Status == "none"
      else "2" if Model_Status == "origin"
      else "3"
    ),
    "sdxl": Sdxl,
    "need_yaml": NeedYaml,
    #"need_yaml": (Model_Status in ["none", "origin"] and Sdxl),
    "need_pipe": PipeInstall,
    "submode": "2" if SubMode == "i2i" else "1",
    "prompt": Prompt,
    "negative_prompt": Negative_Prompt,
    "num": Num,
    "steps": Steps,
    "guidance": Guidance,
    "url": Model,
    "lora_path": LoraPath.strip() if LoraPath.strip() else None
  }
  if SubMode == "i2i":
    civitai_params["strength"] = Strength
    civitai_params["image_path"] = ImagePath

# --- Token数確認 ---
elif mode == "3":
  token_prompt = Prompt

# --- ダウンロード ---
elif mode == "4":
  print("ダウンロード処理を実行します。")

# --- 削除 ---
elif mode == "5":
  print("削除処理を実行します。")



In [ ]:
#@title 実行する
#@markdown ##初回はライブラリのインストールに時間がかかります。<br>
#@markdown **特に、Kindが「1」または「2」での初回の実行では、<br>
#@markdown 以下の場合に時間を要する可能性があります。**
#@markdown - ModelStatus: 「none」または「origin」
#@markdown - NeedYaml: True
#@markdown - PipeInstall: True<br>

#@markdown *加えて、初回以降でもKindが「1」または「2」では、<br>
#@markdown 画像の生成に時間がかかる可能性があります。<br>
#@markdown 異常に実行時間が長い場合は以下の事項を確認してください。*
#@markdown - Model等のパスが正しいか
#@markdown - Modelや画像の容量が大きすぎないか(Model:最大7~8GBあたり)
#@markdown - Stepsが多すぎないか(Stepは通常50~80程度)
#@markdown - Errorが出ていないか
#@markdown - 入力すべき欄で書き忘れがないか
#@markdown - GPUを使用しているか
#@markdown - 回線が低速でないか
#@markdown - 容量(VRAM等)が不足していないか

#@markdown また、SDXLを使用すると長くなる傾向にあります。

model_list_H = ['stablediffusionapi/eleet-model',
              'stablediffusionapi/brav6',
              'Vsukiyaki/ShiratakiMix',
              'stabilityai/stable-diffusion-xl-base-1.0'
]

model_list_C = [["https://civitai.com/api/download/models/28100?type=Model&format=SafeTensor&size=full&fp=fp16","Anime Pastel Dream"],
                ["https://civitai.com/api/download/models/1410435?type=Model&format=SafeTensor&size=pruned&fp=fp16","WAI-NSFW-illustrious-SDXL v:11.0"]
]

#drive
"""
print("Google Driveを使用すると")
print("画像が一枚生成されるごとに保存されます。")
drive_use = input("Google Driveを使用しますか？ (y/n): ").strip().lower() == "y"
if drive_use:
  from google.colab import drive
  import os

  # マウントポイント
  mount_point = '/content/drive'

  # Driveがマウントされているか確認
  if not os.path.exists(mount_point):
    print("Google Driveに接続されていません。接続しますか？ (y/n)")
    connect_drive = input().strip().lower()
    if connect_drive == 'y':
      drive.mount(mount_point)
      print("Google Driveに接続しました。")
    else:
      print("Google Driveへの接続をスキップしました。")
      drive_use = False

if drive_use:
  print("保存する「ディレクトリ」のパスを入力してください。")
  print("(例: /content/drive/MyDrive/Colab Notebooks/your_dir)")
  print("※指定したディレクトリに追加で階層が生成されます(timestamp)。")
  print("保存しない場合は空欄で")
  drive_save = input("Path or Empty: ")
  if drive_save == "":
    drive_use = False

# ---モード選択---
print("=== モード選択 ===")
print("1: Hugging Faceで画像生成")
print("2: CivitAI（モデルDL / safetensors変換）")
print("3: トークン数を数える")
print("4: 出力フォルダをダウンロード(.zip)")
print("5: 出力フォルダを削除")
mode = input("番号を入力してください: ").strip()

# --- 必要なパラメータを先に全て入力 ---
hf_params = {}
civitai_params = {}





if mode == "1":
  hf_params["need_pipe"] = input("pipelineをインストールしますか？ (y/n): ").strip().lower() == "y"
  if hf_params["need_pipe"]:
    hf_params["model_id"] = input("HuggingFaceのモデルIDを入力（例: runwayml/stable-diffusion-v1-5）: ").strip()
  hf_params["submode"] = input("生成モードを選択（1: text2img, 2: img2img）: ").strip()
  hf_params["prompt"] = input("プロンプトを入力: ").strip()
  hf_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  hf_params["num"] = int(input("生成する画像の枚数 : "))
  hf_params["steps"] = int(input("ステップ数（例: 70）: "))
  hf_params["guidance"] = float(input("guidance scale（例: 7.5）: "))
  if hf_params["submode"] == "2":
    hf_params["strength"] = float(input("strength (例: 0.75) : "))
    hf_params["image_path"] = input("初期画像のパスを入力してください（img2img用）: ")
"""
"""
  while True:
    print("修正する入力がある場合は")
    print("num:10")
    print("のように入力してください。")
    print("ない場合は空欄")
    corr = input("入力: ")
    if corr == "":
      break
    corr = corr.split(":")
    if corr[0] in hf_params:
      hf_params[corr[0]] = corr[1]
    elif corr[0] == "drive_use" and not corr[1] == "y":
      drive_use = False
    elif corr[0] == "drive_use" or corr[0] == "drive_save":
      from google.colab import drive
      import os
      # マウントポイント
      mount_point = '/content/drive'
      drive_use = True
      # Driveがマウントされているか確認
      if not os.path.exists(mount_point):
        print("Google Driveに接続されていません。接続しますか？ (y/n)")
        connect_drive = input().strip().lower()
        if connect_drive == 'y':
          drive.mount(mount_point)
          print("Google Driveに接続しました。")
        else:
          print("Google Driveへの接続をスキップしました。")
          drive_use = False
      if corr[1] == "":
        drive_use = False
      if corr[0] == "drive_use":
        print("保存する「ディレクトリ」のパスを入力してください。")
        print("(例: /content/drive/MyDrive/Colab Notebooks/your_dir)")
        print("※指定したディレクトリに追加で階層が生成されます(timestamp)。")
        print("保存しない場合は空欄で")
        drive_save = input("Path or Empty: ")
        if drive_save == "":
          drive_use = False
        else:
          drive_use = True

      if corr[0] == "drive_save" and drive_use:
        drive_save = corr[1]
"""
"""
elif mode == "2":
  print("--- CivitAI モデルの状況は？ ---")
  print("1: まだ何もしていない（DLも変換もしていない）")
  print("2: safetensorsファイルのみアップロード済み（変換が必要）")
  print("3: 変換済みでそのまま使える")
  civitai_params["status"] = input("番号を入力してください: ").strip()
  #civitai_params["model_name"] = input("保存先モデル名（例: models）: ").strip()
  if civitai_params["status"] == "1":
    for k in range(len(model_list_C)):
      print(f"{k+1} : {model_list_C[k][1]}")
    civitai_params["url"] = input("CivitAIのモデル/LoRA ダウンロードURL: ").strip()
    civitai_params["sdxl"] = input("モデルはSDXLですか？ (y/n): ").strip().lower() == "y"
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
  elif civitai_params["status"] == "2":
    print("モデルは")
    print("「/content/models/downloaded_model.safetensors」")
    print("に保存してください。")
    civitai_params["sdxl"] = input("モデルはSDXLですか？ (y/n): ").strip().lower() == "y"
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
  else:
    civitai_params["sdxl"] = input("モデルはSDXLですか？ (y/n): ")
  civitai_params["need_pipe"] = input("pipelineをインストールしますか？ (y/n): ").strip().lower() == "y"
  civitai_params["submode"] = input("生成モードを選択 (1: text2img, 2: img2img) : ").strip()
  civitai_params["prompt"] = input("プロンプトを入力してください: ")
  civitai_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  civitai_params["num"] = int(input("生成する画像の枚数 : "))
  civitai_params["steps"] = int(input("ステップ数（例: 70）: "))
  if civitai_params["submode"] == "2":
    civitai_params["strength"] = float(input("strength (例: 0.75)(img2img用) : "))
    civitai_params["image_path"] = input("初期画像のパスを入力してください（例: /content/your_img.png）: ")
  civitai_params["lora_path"] = input("LoRAを使用する場合はpathを(使わない場合は無入力)")
  if civitai_params["lora_path"] == "":
    civitai_params["lora_path"] = None


elif mode == "3":
  token_prompt = input("プロンプトを入力してください（トークン数確認）: ").strip()
"""

if mode == "4":
  print("少々お待ちください。")

elif mode == "5":
  confirm_delete = input("本当にoutputフォルダを削除しますか？ (y/n): ").strip().lower() == "y"



# --- 必要なinstallはここで実行（全てのinput終了後） ---
if mode in ["1","2","3"] and NeedInstall:
  print("インストールを開始します...")
  !pip install diffusers transformers accelerate safetensors


# --- 各モード実行 ---
if mode == "1":
  from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
  from PIL import Image
  import torch
  import os
  from datetime import datetime, timedelta
  jst = datetime.utcnow() + timedelta(hours=9)
  timeDir = {
      "y":jst.strftime("%Y"),
      "m":jst.strftime("%m"),
      "d":jst.strftime("%d"),
      "t":jst.strftime("%H-%M-%S")
      }
  timestamp = jst.strftime("%Y%m%d_%H%M%S")
  torch_dtype = torch.float16

  if 'model_id' not in hf_params:
    hf_params["model_id"] = "unknown"
  if hf_params["need_pipe"]:
    if hf_params["submode"] == "1":
      pipe = StableDiffusionPipeline.from_pretrained(
        hf_params["model_id"],
        torch_dtype=torch.float16,
        safety_checker=None  # Safe checkを無効に
      )
      pipe = pipe.to("cuda")  # GPUが必要です
    else:
      pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        hf_params["model_id"],
        torch_dtype=torch.float16,
        safety_checker=None
      )
      pipe = pipe.to("cuda")
  if hf_params["submode"] != "1":
    init_image = Image.open(hf_params["image_path"]).convert("RGB").resize((512, 512))
  for k in range(hf_params["num"]):
    if hf_params["submode"] == "1":

      image = pipe(hf_params["prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   guidance_scale=hf_params["guidance"],
                   added_cond_kwargs={}).images[0]
    else:
      #init_image = Image.open(hf_params["image_path"]).convert("RGB").resize((512, 512))
      image = pipe(prompt=hf_params["prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   image=init_image,
                   strength=hf_params["strength"],
                   guidance_scale=hf_params["guidance"]).images[0]
    if drive_use:
      save_dir = os.path.join(
          drive_save,
          timeDir["y"]+"-"+timeDir["m"],
          timeDir["d"],
          timeDir["t"]
      )

      os.makedirs(save_dir, exist_ok=True)
      image.save(save_dir+"/"+str(k+1)+".png")

      # プロンプトとネガティブプロンプトを保存（1回目のみ）
      if k == 0:
        with open(os.path.join(save_dir, "info.txt"), "w", encoding="utf-8") as f:
          if hf_params["submode"] == "1":
            f.write("t2i\n\n")
          else:
            f.write("i2i\n\n")
          f.write("Model:\nHugging Face / "+hf_params["model_id"]+"\n\n")
          f.write("Prompt:\n" + hf_params["prompt"] + "\n\n")
          f.write("Negative Prompt:\n" + hf_params["negative_prompt"])

    else:
      os.makedirs("output/"+timestamp+"/"+hf_params["prompt"][0:10], exist_ok=True)
      filename = f"output/"+timestamp+"/"+hf_params["prompt"][0:10]+"/"+str(k+1)+".png"
      image.save(filename)
  print("画像を保存しました")

elif mode == "2":
  import os
  import torch
  import re
  from datetime import datetime, timedelta
  jst = datetime.utcnow() + timedelta(hours=9)
  timeDir = {
      "y":jst.strftime("%Y"),
      "m":jst.strftime("%m"),
      "d":jst.strftime("%d"),
      "t":jst.strftime("%H-%M-%S")
      }
  timestamp = jst.strftime("%Y%m%d_%H%M%S")
  if civitai_params["submode"] == "2":
    from PIL import Image
    from IPython.display import display


  base_dir = "/content/models"
  os.makedirs(base_dir, exist_ok=True)
  download_path = "/content/models/downloaded_model.safetensors"

  #※Driveのinfo.txtに書くための変数!!
  modelName = "unknown"
  if "Model" in locals():
    regex = r'([0-9]+)'
    regex2 = r'#(\d+)'
    if bool(re.match(regex, Model)):
      modelName = model_list_C[int(Model)-1][1]
    else:
      match = re.fullmatch(regex2, Model)
      if match:
        modelName = "model_id:" + match.group(1)
      else:
        modelName = Model


  if civitai_params["status"] == "1":
    regex = r'([0-9]+)'
    regex2 = r'#(\d+)'
    if bool(re.match(regex, civitai_params['url'])):
      civitai_params['url'] = model_list_C[int(civitai_params['url'])-1][0]
    else:
      match = re.fullmatch(regex2, civitai_params['url'])
      if match:
        civitai_params['url'] = 'https://civitai.com/api/download/models/'+match.group(1)

    model_url = civitai_params["url"]

    print(model_url)

    print(f"Downloading Checkpoint model to {download_path}...")
    !wget -O {download_path} "{model_url}"
    print(f"Download of safetensorFile complete!")

    # Checkpointモデルのパスを記録
    checkpoint_path = download_path

  lora_path = civitai_params["lora_path"] # LoRAはダウンロードしないとNone


  if civitai_params["status"] == "1" or civitai_params["status"] == "2":
    if civitai_params["need_yaml"]:
      !wget -O /content/models/v1-inference.yaml \
https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
      #!wget -q https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-inference.yaml
    if not civitai_params["sdxl"]:
      !mkdir /content/converted
      !wget -O convert_original_stable_diffusion_to_diffusers.py \
      https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
      os.makedirs(base_dir, exist_ok=True)
      !python convert_original_stable_diffusion_to_diffusers.py \
        --checkpoint_path ./models/downloaded_model.safetensors \
        --original_config_file ./models/v1-inference.yaml \
        --dump_path ./converted/diffusers_model \
        --from_safetensors

  #pipe = None
  if civitai_params["need_pipe"]:
    if civitai_params["sdxl"]:
      if civitai_params["submode"] == "1":
        from diffusers import StableDiffusionXLPipeline
        pipe_civitai = StableDiffusionXLPipeline.from_single_file(
            download_path,
            torch_dtype=torch.float16, # fp16モデルなので必須
            use_safetensors=True,      # safetensors形式であることを明示
            safety_checker=None
        ).to("cuda")
      else:
        from diffusers import StableDiffusionXLImg2ImgPipeline
        from PIL import Image
        import torch

        pipe_civitai = StableDiffusionXLImg2ImgPipeline.from_single_file(
            download_path,
            torch_dtype=torch.float16,
            use_safetensors=True,
            safety_checker=None,
        ).to("cuda")

      # VRAM節約と高速化のための設定（非常に重要）
      #pipe.enable_xformers_memory_efficient_attention()

      # 最も重要なVRAM節約策：モデルの一部をCPUにオフロード
      # VRAMが16GB以下のGPUでは、SDXLを動かすためにほぼ必須です
      pipe_civitai.enable_model_cpu_offload()
    else:
      from diffusers import StableDiffusionPipeline
      #pipe = StableDiffusionPipeline.from_pretrained(base_dir, torch_dtype=torch.float16).to("cuda")
      pipe_civitai = StableDiffusionPipeline.from_pretrained(
          "/content/converted/diffusers_model",
          torch_dtype=torch.float16,
          use_safetensors=False,
          safety_checker=None
        ).to("cuda")
    # パイプラインを作成した直後に以下を追加します
    if lora_path:
      pipe_civitai.load_lora_weights(lora_path, use_safetensors=True)


  if civitai_params["submode"] == "1":
  #pipe.enable_xformers_memory_efficient_attention()
    for k in range(civitai_params["num"]):
      image = pipe_civitai(prompt=civitai_params["prompt"],
          negative_prompt=civitai_params["negative_prompt"],
          num_inference_steps=civitai_params["steps"]
      ).images[0]
      if drive_use:
        save_dir = os.path.join(
            drive_save,
            timeDir["y"]+"-"+timeDir["m"],
            timeDir["d"],
            timeDir["t"]
          )
        os.makedirs(save_dir, exist_ok=True)
        image.save(save_dir+"/"+str(k+1)+".png")

        # プロンプトとネガティブプロンプトを保存（1回目のみ）
        if k == 0:
          with open(os.path.join(save_dir, "info.txt"), "w", encoding="utf-8") as f:
            f.write("t2i\n\n")
            if civitai_params["sdxl"]:
              f.write("Model:\nCivitAI(SDXL) / "+modelName+"\n\n")
            else:
              f.write("Model:\nCivitAI / "+modelName+"\n\n")
            f.write("Prompt:\n" + civitai_params["prompt"] + "\n\n")
            f.write("Negative Prompt:\n" + civitai_params["negative_prompt"])

      else:
        os.makedirs("output/t2i/"+timestamp+"/"+civitai_params["prompt"][0:10], exist_ok=True)
        image.save("output/t2i/"+timestamp+"/"+civitai_params["prompt"][0:10]+"/"+str(k+1)+".png")
    print("画像を保存しました: output/t2i/"+timestamp+"/"+civitai_params["prompt"][0:10])
  else:
    init_image = Image.open(civitai_params["image_path"]).convert("RGB").resize((1024, 1024))
    #display(init_image)
    for k in range(civitai_params["num"]):
      image = pipe_civitai(prompt=civitai_params["prompt"],
          negative_prompt=civitai_params["negative_prompt"],
          image=init_image,
          num_inference_steps=civitai_params["steps"],
          strength=civitai_params["strength"],
      ).images[0]
      if drive_use:
        save_dir = os.path.join(
            drive_save,
            timeDir["y"]+"-"+timeDir["m"],
            timeDir["d"],
            timeDir["t"]
          )
        os.makedirs(save_dir, exist_ok=True)
        image.save(save_dir+"/"+str(k+1)+".png")

        # プロンプトとネガティブプロンプトを保存（1回目のみ）
        if k == 0:
          with open(os.path.join(save_dir, "info.txt"), "w", encoding="utf-8") as f:
            f.write("i2i\n\n")
            if civitai_params["sdxl"]:
              f.write("Model:\nCivitAI(SDXL) / "+modelName+"\n\n")
            else:
              f.write("Model:\nCivitAI / "+modelName+"\n\n")
            f.write("Prompt:\n" + civitai_params["prompt"] + "\n\n")
            f.write("Negative Prompt:\n" + civitai_params["negative_prompt"])
      else:
        os.makedirs("output/i2i/"+timestamp+"/"+civitai_params["prompt"][0:10], exist_ok=True)
        image.save("output/i2i/"+timestamp+"/"+civitai_params["prompt"][0:10]+"/"+str(k+1)+".png")
    print("画像を保存しました: output/i2i/"+timestamp+"/"+civitai_params["prompt"][0:10])

elif mode == "3":
  from transformers import CLIPTokenizer
  tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
  tokens = tokenizer(token_prompt)["input_ids"]
  print(f"トークン数: {len(tokens)}")

elif mode == "4":
  import shutil
  from google.colab import files

  # 圧縮したいディレクトリのパス
  folder_path = '/content/output'
  zip_filename = folder_path + '.zip'

  # ディレクトリを zip 形式で圧縮
  shutil.make_archive(folder_path, 'zip', folder_path)

  # Colab にダウンロードリンクを表示
  files.download(zip_filename)


elif mode == "5":
  import shutil
  if confirm_delete and os.path.exists("output"):
    shutil.rmtree("output")
    print("outputフォルダを削除しました")
  else:
    print("キャンセルされました")

elif mode == "6":
  default_params = {
    "Drive_path": "",
    "Kind": "Hugging Face",
    "SubMode": "t2i",
    "Prompt": "",
    "Negative_Prompt": "",
    "Num": 10,
    "Steps": 50,
    "ImagePath": "",
    "Strength": 0.75,
    "Guidance": 7.5,
    "Model_Status": "none",
    "Model": "2",
    "Sdxl": True,
    "PipeInstall": True,
    "LoraPath": ""
  }

  # 初期化関数
  for k in default_params.keys():
    if type(default_params[k]) == int or type(default_params[k]) == float:
      print(k+" : "+str(default_params[k]))
    elif type(default_params[k]) == bool:
      print(k+" : "+str(default_params[k]))
    else:
      print(k+' : "'+default_params[k]+'"')




  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]